In [2]:
import numpy as np
import pandas as pd
import hashlib

df= pd.read_excel("fanta.xlsx")
#df.drop(columns=['Team 1', 'Team 2','Team 3', 'Team 4','Team 5', 'Team 6','Team 7', 'Team 8'])

#classes
class League:
    pw='password'
    def __init__(self, name):
        self.name = name
        self.team_Names=[]
        self.team_array=[]

    def addTeam(self,team):
        if team.name not in self.team_Names:
            self.team_Names.append(team.name)
            lista_appoggio = []
            for k in range(df.shape[0]):
                lista_appoggio.append(0)
            df[team.name]= lista_appoggio
            self.team_array.append(team)
        else:
            print('Devi cambiare il nome ')
        
    def printResult(self):
        classifica=self.team_array
        print('Classifica della lega',self.name)
        i=1
        while classifica:
            massimo=0
            for team in classifica:
                if team.getTotalPoints()>= massimo:
                    massimo = team.getTotalPoints()
                    team_max = team
            print(i, 'posto :', team_max.name, 'con', massimo, 'punti' )
            i= i+1
            classifica.remove(team_max)
            
    def changePw(self,password):
        self.pw=password
            
class Team:
    points=[0,0,0,0,0]
    #formation=[Squad,Squad,Squad,Squad,Squad]
    def __init__(self, name):
        self.name = name
        self.points = [0,0,0,0,0]
        self.formation=[Squad,Squad,Squad,Squad,Squad]
        
    def addPoints(self, point, giornata):
        self.points[giornata]=point
    def getPoints(self):
        print(self.points)
    def getTotalPoints(self):
        i=0
        somma=0
        for i in range(5):
            somma = somma+self.points[i]
        return somma
    
    
    
class Player:
    roles=[]
    
    def __init__(self, name):
        self.name = name
        if list(np.where(df["Nome"]== self.name)[0]):
            indice = np.where(df["Nome"]== self.name)[0][0]
            self.roles=df['RM'][indice].split(";")
            self.exists=True
        else:
            print('Il giocatore non esiste o non è più in lista')
            self.exists=False
            
    def checkIfCanBeUsed(self, team):
        indice = np.where(df["Nome"]== self.name)[0][0]
        if df[team.name][indice]==0:
            return True
        else:
            return False
        
    def setUsed(self,team):
        indice = np.where(df["Nome"] == self.name)[0][0]
        df.at[indice,team.name] = 1 
        
    def getRoles(self):
        #print(self.roles)
        return self.roles
    def checkExistance(self):
        return self.exists
    
    
    
class Squad:
    
    starting_eleven=[]
    bench_players=[]
    giornata=0
    
    def __init__(self, modul):
        self.starting_eleven=[]
        self.bench_players=[]
        self.module = modul
        
    def addStartingEleven(self, pos, starting_player,team):
        if self.module.ruoli:
            if starting_player.checkIfCanBeUsed(team):
                if self.module.addPlayer(pos,starting_player)!=-1:
                        self.starting_eleven.append(starting_player)
                        starting_player.setUsed(team)
                else:
                    None
            else:
                print('Hai già schierato', starting_player.name,'in precedenza')
        else:
            print('La formazione è al completo, non posso inserire', starting_player.name)
    
    def addBenchPlayer(self,bench_player,team):
        if bench_player.checkIfCanBeUsed(team):        
            if len(self.bench_players) < 9 :
                self.bench_players.append(bench_player)
                bench_player.setUsed(team)
            else:
                None
        else:
            print('Hai già schierato', bench_player.name,'in precedenza')
            
    def checkNotSameFootballTeam(self,giocatore):
        i=0
        j = np.where(df["Nome"]== giocatore.name)[0][0]
        for footballer in self.starting_eleven:
            i = np.where(df["Nome"]== footballer.name)[0][0]
            if df['Squadra'][i]== df['Squadra'][j]:
                print('Hai già usato un giocatore della squadra', df['Squadra'][j])
                return False
        i=0
        for footballer in self.bench_players:
            i = np.where(df["Nome"]== footballer.name)[0][0]
            if df['Squadra'][i]== df['Squadra'][j]:
                print('Hai già usato un giocatore della squadra', df['Squadra'][j])
                return False
        return True
    
    #def checkName(self,giocatore):
     #   if list(np.where(df["Nome"]== giocatore.name)[0]):
      #      return True
       # else:
        #    print(giocatore.name,'non è nella lista, ricontrolla')
         #   return False
        

class Module:
    ruoli=[]
    def __init__(self,name):
        if name=='343':
            self.ruoli=['Por','Dc','Dc','Dc','E','E','C','M;C','W;A','W;A','A;Pc']
        elif name=='3412':
            self.ruoli=['Por','Dc','Dc','Dc','E','E','C','M;C','T','A;Pc','A;Pc']
        elif name=='3421':
            self.ruoli=['Por','Dc','Dc','Dc','E','E;W','C','M;C','T','T;A','A;Pc']
        elif name=='352':
            self.ruoli=['Por','Dc','Dc','Dc','E','E;W','C','M','M;C','A;Pc','A;Pc']
        elif name=='3511':
            self.ruoli=['Por','Dc','Dc','Dc','E;W','E;W','C','M','M','T;A','A;Pc']
        elif name== '433':
            self.ruoli=['Por','Dd','Dc','Dc','Ds','M','M;C','C','W;A','W;A','A;Pc']
        elif name=='4312':
            self.ruoli=['Por','Dd','Dc','Dc','Ds','M','M;C','C','T','A;Pc;T','A;Pc']
        elif name=='442':
            self.ruoli=['Por','Dd','Dc','Dc','Ds','M;C','C','E','W;E','A;Pc','A;Pc']
        elif name=='4141':
            self.ruoli=['Por','Dd','Dc','Dc','Ds','M','C;T','T','W','W;E','A;Pc']
        elif name=='4411':
            self.ruoli=['Por','Dd','Dc','Dc','Ds','M','C','E;W','W;E','A;T','A;Pc']
        elif name=='4231':
            self.ruoli=['Por','Dd','Dc','Dc','Ds','M;C','M','T','W;T','A','A;Pc']
            
    def checkPos(self,pos,player_to_insert):
        for ruolo in player_to_insert.roles:
            if ruolo in self.ruoli[pos].split(";"):
                return pos
        return -1
    
    def addPlayer(self, pos,player_to_insert):
        if self.ruoli:
            i=self.checkPos(pos,player_to_insert)
            if i==-1:
                print('Non puoi inserire questo giocatore in questa posizione. Il giocatore non possiede questo ruolo')
                return i
            else:
                self.ruoli.pop(pos)
                print(self.ruoli)
                return i
        else:
            None
    def getRuoli(self):
        print(self.ruoli)

In [ ]:
#list_of_league=[]
#lega=League('Prova')
#list_of_league.append(lega)

process_is_not_finished=True
stato='Enter League'
list_of_league=[]

while process_is_not_finished:
    
    if stato=='Enter League':
        nome_lega = input('Inserisci il nome della Lega: ')
        if list_of_league:
            for lista in list_of_league:
                if lista.name==nome_lega:
                    stato='Accedi o Registrati'
                    #print('ok')
                    break
                else:
                    stato='Create League'
        else:
            lega_scelta=League(nome_lega)
            list_of_league.append(lega_scelta)
            stato='Accedi o Registrati' 
                
    if stato=='Create League':
        lega_scelta=League(nome_lega)
        list_of_league.append(lega_scelta)
        stato='Accedi o Registrati'
    
    if stato=='Accedi o Registrati':
        i= input('1. Login\n2. Registrati ')
        if i=='1':
            stato='Login'
        elif i=='2':
            stato='Registrati'
    
    if stato=='Login':
        team_name = input('Enter Team Name: ')
        pwd = input('Enter password: ')
        auth = pwd.encode()
        auth_hash = hashlib.md5(auth).hexdigest()
        with open('credentials.txt', 'r') as f:
            stored_name, stored_pwd = f.read().split('\n')
        f.close()
        if team_name == stored_name and auth_hash == stored_pwd:
            print('Accesso eseguito correttamente!')
            is_ok=True
        else:
            print('Login fallito! \n')
            is_ok= False
            
        for lista in list_of_league:
            if lista.name==nome_lega:
                lista.addTeam(Team(team_name))
                #lista.printResult()
                      
        if is_ok=='False':
            stato='Login'
        else:
            stato='Gestisci Lega'
            
    
    if stato=='Gestisci Lega':
        che_fare= input('1. Accedi come admin\n2. Scegli Modulo\n')
        if che_fare=='1':
            passwd=input('Inserisci pw per la Lega: ')
            if passwd==lega_scelta.pw:
                stato='Amministra'
            else:
                print('Password errata!')
        elif che_fare=='2':
            stato='Scegli Modulo'
    
    #DA FINIRE DA FINIRE DA FINIRE DA FINIRE
    if stato=='Amministra':
        azione= input('1. Calcola giornata\n2. Cambia password di Lega\n3. Torna Indietro\n')
        if azione=='1':
            giorno= int(input('Scegli giornata:'))
            for team in lega_scelta.team_array:
                print('Punti fatti dal team:', team.name,'nella giornata', str(giorno))
                punti = input()
                team.addPoints(punti,giorno)
            # DA FINIRE DA FINIRE lega_scelta.printResult()
            stato='Gestisci Lega'
        if azione=='2':
            new_pw=input('Scegli la nuova password della lega')
            lega_scelta.changePw(new_pw)
        if azione=='3':
            stato='Gestisci Lega'
    
        
    
    if stato=='Registrati':
        team_name = input('Enter Team Name: ')
        pwd = input('Enter password: ')
        conf_pwd = input('Confirm password: ')
        if conf_pwd == pwd:
            enc = conf_pwd.encode()
            hash1 = hashlib.md5(enc).hexdigest()
            with open('credentials.txt', 'w') as f:
                f.write(team_name + '\n')
                f.write(hash1)
            f.close()
            print('Registrazione effettuata !')
            is_ok=True
        else:
            print('Password non coincidono! Riprova \n')
            is_ok=False
        
        for lista in list_of_league:
            if lista.name==nome_lega:
                team_scelto = Team(team_name)
                lista.addTeam(team_scelto)
                
                #lista.printResult()
        if is_ok=='False':
            stato='Registrati'
        else:
            stato='Gestisci Lega'
    
    if stato=='Inserisci Formazione':
        print(modulo_scelto.ruoli)
        not_completed= True
        sub_stato='Scegli Titolari'
        formazione_scelta= Squad(modulo_scelto)
        
        while not_completed:
            if sub_stato=='Scegli Titolari':
                for play in formazione_scelta.starting_eleven:
                    print(play.name)
                posizione= int(input('Scegli la posizione da inserire: '))
                giocatore= input('Scegli il giocatore titolare: ')
                giocatore_scelto=Player(giocatore)
                if formazione_scelta.checkNotSameFootballTeam(giocatore_scelto):
                    formazione_scelta.addStartingEleven(posizione,giocatore_scelto,team_scelto)
                if not formazione_scelta.module.ruoli:
                    sub_stato='Scegli Panchina' 
            if sub_stato=='Scegli Panchina':
                giocatore= input('Scegli il giocatore in panchina:')
                tempo_player= Player(giocatore)
                if formazione_scelta.checkNotSameFootballTeam(tempo_player):
                    formazione_scelta.addBenchPlayer(tempo_player,team_scelto)
                if len(formazione_scelta.bench_players)==3:
                    not_completed=False
        
        stato='Conferma Finale'
                
                
                
        
    if stato=='Scegli Modulo':
        scelta= input('Scegli modulo')
        modulo_scelto= Module(scelta)
        if not modulo_scelto.ruoli:
            stato='Scegli Modulo'
            print('Hai inserito un modulo che non esiste')
        else:
            stato='Inserisci Formazione'
        
        
    if stato=='Conferma Finale':
        for play in formazione_scelta.starting_eleven:
            print(play.name)
        for play in formazione_scelta.bench_players:
            print(play.name)
        conferma= input('Vuoi proseguire:\n1. Sì\n2. No\n ')
        if conferma== '1':
            process_is_not_finished = False
        elif conferma=='2':
            process_is_not_finished=True
            stato=''
    

print('Hai inserito la formazione adesso puoi giocare')
        
            